In [1]:
# Generamos un DataFrame con las columnas Vup Iup Vdw Idw

# Update Oct 24
# Nombre archivos: celda1_ciclo_0000.txt     

import pandas as pd
import numpy as np
import sys

# Leer el archivo de texto con pandas
def leer_txt_pandas(filepath):
    # Leer el archivo como texto y eliminar las primeras líneas de encabezado
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Extraer la fecha y hora
    date = lines[3].strip()  # Fecha en la línea 4
    time = lines[4].strip()  # Hora en la línea 5
    
    # Crear columna datetime
    datetime = pd.to_datetime(f"{date} {time}", format="%d-%m-%Y %H:%M:%S")
    
    # Leer datos de DIR UP (líneas 7 a 56) y DIR DOWN (líneas 63 a 113)
    df_up = pd.read_csv(filepath, sep=r'\s+', skiprows=6, nrows=50, names=["Vup", "Iup"])
    df_down = pd.read_csv(filepath, sep=r'\s+', skiprows=63, nrows=50, names=["Vdw", "Idw"])
    
    # Agregar la columna datetime
    df_up['datetime'] = datetime
    #df_down['datetime'] = datetime

    #setting datetime to index is done inside the loop

    # Combinar ambos DataFrames
    df_final = pd.concat([df_up.reset_index(drop=True), df_down.reset_index(drop=True)], axis=1)

    return df_final     #, ciclo, rep])



In [2]:
## Loop para crear n dfs correspondientes a los primeros n ciclos de m celdas

num_celda = ['1','2']           #nombre celdas
#n = 80                          #cantidad de ciclos
n = 1436

lista_df_celda=[]
for j in num_celda:                         #loop sobre las celdas
    lista_df=[]
    for i in range(n):                      #loop sobre los ciclos
        ciclo = f'{i:04d}'                  #formato según el nombre de los archivos de SplitData

        filepath = 'SplitData/celda'+j+'_ciclo_'+ ciclo + '.txt'               #cambiar a carpeta split2    

        # Llamar la función para procesar el archivo
        df1 = leer_txt_pandas(filepath)                            #df1 , ciclo , rep = leer_txt_pandas(filepath)   

        # Poner la columna datetime como index      
        df1.set_index('datetime', inplace=True)     

        num_rows = len(df1)         
        increments = pd.to_timedelta(4 * np.arange(0, len(df1) * 0.01, 0.01), unit='s')         # Revisar incremento
        df1.index = df1.index + increments  # Sumar los incrementos al índice     

        # Convertir las las ',' por '.' 
        df1[['Vup', 'Iup', 'Vdw', 'Idw']] = df1[['Vup', 'Iup', 'Vdw', 'Idw']].replace(',', '.', regex=True)

        # Convertir las columnas 'Vup', 'Iup', 'Vdw' e 'Idw' a tipo float
        df1[['Vup', 'Iup', 'Vdw', 'Idw']] = df1[['Vup', 'Iup', 'Vdw', 'Idw']].astype(float)

        lista_df.append(df1)
    lista_df_celda.append(lista_df)    

#lista_df_celda[0][0]   
    

In [3]:
#Import data from local station (UV)
aws_df = pd.read_csv('aws_cienciasUV_start_2024_11_06_17_51_00_UTC_1.csv').set_index('Date')
aws_df.index = pd.to_datetime(aws_df.index, format='%m/%d/%y %H:%M:%S')         #datetime index

#Rename columns                        ## Units: C, %, grados, m/s, mbar, W/m^2, mm
cols = aws_df.columns
aws_df = aws_df.rename(columns={cols[1]:"Temperature", cols[2]: "rh", cols[3]:"DewPoint", cols[4]:"WindDir", cols[5]: "WindSpeed",
                                cols[6]:"GustSpeed", cols[7]:"Pressure", cols[8]:"SolarRad", cols[9]:"Rain"})

#Convertir a Nan los datos raros (checkear estos errores con Omar -> Falló sensor por tormenta)
aws_df = aws_df.applymap(lambda x: np.nan if isinstance(x, str) and ',' in x else x)

#SolarRad To float
aws_df[['SolarRad']] = aws_df[['SolarRad']].astype(float)  

# Define piranometer df
Pin_df = aws_df['SolarRad'].to_frame()
#Pin_df = Pin_df.loc[Pin_df.index > pd.Timestamp("16-10-2024")]                  #Descartar los valores que no entran en la fecha deseada


In [4]:
# Function to add Pin column to nearest index on df 

def addPin(df, Pin_df):
    left = df.reset_index()
    right = Pin_df.reset_index('Date')
    merged_df = pd.merge_asof(left, right, on='Date', direction='nearest')

    return merged_df.set_index('Date')         # Set the index back to datetime

# Functions for Voc, JSC, Jmax and Vmax

def ocsc_old(df, varname, dir):    
    '''Returns corresponding Isc or Voc depending on variable and direction'''
    var = varname + dir
    current = 'I' + dir
    notvar = next(filter(lambda x: x != varname, ['I', 'V']), None) + dir
    if var == 'Vdw':
        return np.abs(df[var].loc[(df[current] > 0) & (df[notvar].abs() == df[notvar].abs().loc[df[current] > 0].min())][0]) 
    elif var == 'Idw':
        return np.abs(df[var].loc[(df[current] < 0) & (df[notvar].abs() == df[notvar].abs().loc[df[notvar] > 0].min())][0]) 
    else: 
        return np.abs(df[var].loc[(df[current] < 0) & (df[notvar].abs() == df[notvar].abs().loc[df[current] < 0].min())][0])

def ocsc(df, varname, dir):    
    '''Returns corresponding Isc or Voc depending only on variable'''
    var = varname + dir
    current = 'I' + dir
    notvar = next(filter(lambda x: x != varname, ['I', 'V']), None) + dir
    return np.abs(df[var].loc[(df[current] < 0) & (df[notvar].abs() == df[notvar].abs().loc[df[current] < 0].min())][0])    

def pmax(df, varname, dir):
    '''Return corresponding Imax or Vmax depending only on variable'''
    var = varname + dir
    current = 'I' + dir
    return np.abs(df[var].loc[(df[current].loc[df[current] < 0] * df['V' + dir]).abs().idxmax()])

# Define functions: FF & PCE for each run ; HI for each cicle (i.e. up and down runs)

def FF(Vmax, jmax, Voc, jSC):
    return 100 * Vmax*jmax / (Voc*jSC)                  #  [%] Adimensional

def PCE(Vmax, jmax, Pin=100):                           # Eficiencia
    return 100.0 * np.abs(Vmax * jmax / Pin)            # [%]  ;  Pin: dato piranómetro ; (lab) P_In = 100 mW/cm^2 = 100 V*mA/cm^2

def HI(jMaxFor, jMaxRev):
    return 1.0 + jMaxFor / jMaxRev 

def nearest(index, column, df_ref = aws_df):      # Search for the closest index and outputs corresponding column value
    closest_idx_value = df_ref.index[df_ref.index.get_indexer([index], method='nearest')[0]]
    return df_ref[column].loc[closest_idx_value]


In [5]:
#Hallar Parámetros fotovoltaicos (doble for loop)

pd.options.mode.copy_on_write = True

Ama_list = [1.0, 20.25]

for j in range(len(num_celda)):                    #loop sobre celdas
    lista_df = lista_df_celda[j]                   #lista de df correspondientes a la celda j
    Ama = Ama_list[j]
    data = []
    ind_list = []

    for i in range(n):                             #loop sobre ciclos
        df = lista_df[i]      

        Jsc_up = ocsc(df, 'I', 'up') / Ama * 1e3
        Jsc_dw = ocsc(df, 'I', 'dw') / Ama * 1e3
        Voc_up = ocsc(df, 'V', 'up')
        Voc_dw = ocsc(df, 'V', 'dw')

        Jmax_up = pmax(df, 'I', 'up') / Ama * 1e3
        Jmax_dw = pmax(df, 'I', 'dw') / Ama * 1e3
        Vmax_up = pmax(df, 'V', 'up')
        Vmax_dw = pmax(df, 'V', 'dw')

        # Search for the closest index and outputs corresponding column value
        Pin_dw = nearest(df.index[0],'SolarRad', Pin_df)
        Pin_up = Pin_dw  

        FF_up = FF(Vmax_up, Jmax_up, Voc_up, Jsc_up)
        FF_dw = FF(Vmax_dw, Jmax_dw, Voc_dw, Jsc_dw)
        PCE_up = PCE(Vmax_up, Jmax_up, Pin_up)
        PCE_dw = PCE(Vmax_dw, Jmax_dw, Pin_dw)        
                     
        hi = round(HI(Jmax_up, Jmax_dw),6)

        lista_parametros = [i+1, Voc_dw, Jsc_dw, Jmax_dw, Vmax_dw, PCE_dw, FF_dw,
                            Voc_up, Jsc_up, Jmax_up ,Vmax_up, PCE_up, FF_up, hi]

        data.append(lista_parametros)
        ind_list.append(df.index[0])     
    
    index = pd.DatetimeIndex(ind_list)                              
    df4 = pd.DataFrame(data,index=index, columns=['ciclo','Voc_dw', 'Jsc_dw', 'Jmax_dw','Vmax_dw', 'PCE_dw', 'FF_dw',
                                    'Voc_up', 'Jsc_up', 'JMax_up', 'Vmax_up', 'PCE_up', 'FF_up', 'HI'])
    df4.index.name = 'Date'
    
    df5 = addPin(df4, Pin_df)                 # Add Pin column

    df5.to_pickle('celda'+str(j+1)+'_'+str(n)+'ciclos.pkl')


In [6]:
df5          #Celda 2  (ciclos cada media hora, como debe ser)


,ciclo,Voc_dw,Jsc_dw,Jmax_dw,Vmax_dw,PCE_dw,FF_dw,Voc_up,Jsc_up,JMax_up,Vmax_up,PCE_up,FF_up,HI,SolarRad
Date,,,,,,,,,,,,,,,
2024-10-21 06:00:48,1,0.0100,0.000012,0.000012,0.0100,0.000012,100.000000,0.0100,0.000007,0.000008,0.0455,0.000038,522.607651,1.691785,1.0
2024-10-21 06:30:49,2,0.2665,0.000143,0.000078,0.1005,0.000789,20.656729,0.2110,0.000114,0.000074,0.1005,0.000743,30.848171,1.941693,1.0
2024-10-21 07:00:46,3,2.0915,0.029284,0.021383,1.4835,3.172126,51.792024,2.0915,0.028395,0.020296,1.5385,3.122585,52.579156,1.949192,1.0
2024-10-21 07:30:45,4,2.4235,0.297333,0.219506,1.7600,38.633086,53.613330,2.4235,0.295012,0.218864,1.7600,38.520099,53.877096,1.997075,1.0
2024-10-21 08:00:45,5,2.4790,0.727407,0.531358,1.7600,93.519012,51.861571,2.4790,0.724642,0.530222,1.7600,93.319111,51.948209,1.997862,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 13:00:28,1432,2.5340,3.365037,2.290617,1.5385,0.615029,41.328874,2.5340,3.366222,2.289284,1.5385,0.614671,41.290275,1.999418,573.0
2024-11-06 13:10:28,1433,2.5895,3.355062,2.283160,1.5385,0.584466,40.431282,2.5895,3.350074,2.288642,1.5385,0.585869,40.588690,2.002401,601.0
2024-11-06 13:20:28,1434,2.5340,3.343309,2.271012,1.5385,0.552841,41.241449,2.5340,3.339111,2.271802,1.5385,0.553033,41.307659,2.000348,632.0


In [7]:
# Exportar df combinado: seteamos date a una columna y hacemos el merge según ciclos y celdas

df1 = pd.read_pickle('celda'+str(1)+'_'+str(n)+'ciclos.pkl').reset_index('Date').set_index('ciclo')
df2 = pd.read_pickle('celda'+str(2)+'_'+str(n)+'ciclos.pkl').reset_index('Date').set_index('ciclo')

#print(df1) 

# Create a MultiIndex
arrays = [df1.index.tolist() * 2, ['1'] * len(df1) + ['2'] * len(df2)]
#print(arrays)

df_combined = pd.concat([df1, df2]).sort_index(kind='stable')           # Sorting algorithm -> stable

df_combined.index = pd.MultiIndex.from_tuples(
    [(idx, i) for idx in df_combined.index.unique() for i in range(1, (df_combined.index == idx).sum() + 1)],
    names=['Ciclo', 'Celda'])


#df_combined.iloc[:20:2]                         #celda 1, primeros 10 ciclos

df_combined[['Date', 'Voc_dw', 'Jsc_dw', 'PCE_dw', 'FF_dw', 'Voc_up', 'Jsc_up', 'PCE_up', 'FF_up', 'HI', 'SolarRad']]
#df_combined


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 2

Date  Voc_dw    Jsc_dw    PCE_dw       FF_dw  \
Ciclo Celda                                                               
1     1     2024-10-21 06:00:27  0.8750  0.000861  0.086019  114.120437   
      2     2024-10-21 06:00:48  0.0100  0.000012  0.000012  100.000000   
2     1     2024-10-21 06:30:28  0.8750  0.000677  0.086019  145.178424   
      2     2024-10-21 06:30:49  0.2665  0.000143  0.000789   20.656729   
3     1     2024-10-21 07:00:27  0.8750  0.017820  0.829391   53.191425   
...                         ...     ...       ...       ...         ...   
1434  2     2024-11-06 13:20:28  2.5340  3.343309  0.552841   41.241449   
1435  1     2024-11-06 13:30:10  1.0410  0.944000  0.060901   40.964115   
      2     2024-11-06 13:30:28  2.5340  3.328889  0.526242   41.236461   
1436  1     2024-11-06 13:40:10  1.0410  0.931000  0.057360   40.659543   
      2     2024-11-06 13:40:29  2.5340  3.293333  0.503516   41.450314   

             Voc_up    Jsc_up    PCE_up       FF_up        HI  SolarRad  
Ciclo Celda                                                              
1     1      0.0100  0.000032  0.000032  100.000000  1.030235       1.0  
      2      0.0100  0.000007  0.000038  522.607651  1.691785       1.0  
2     1      0.1005  0.000324  0.000802   24.596923  1.167918       1.0  
      2      0.2110  0.000114  0.000743   30.848171  1.941693       1.0  
3     1      0.8195  0.019505  0.826120   51.683066  2.198751       1.0  
...             ...       ...       ...         ...       ...       ...  
1434  2      2.5340  3.339111  0.553033   41.307659  2.000348     632.0  
1435  1      1.0960  1.192000  0.080132   40.543442  2.436688     661.0  
      2      2.5340  3.323704  0.525528   41.244760  2.034668     661.0  
1436  1      1.0410  1.188000  0.077012   42.780834  2.466003     687.0  
      2      2.5340  3.296889  0.503339   41.391061  1.999649     687.0  

[2872 rows x 11 columns]

In [11]:
df_combined.describe()

,Date,Voc_dw,Jsc_dw,Jmax_dw,Vmax_dw,PCE_dw,FF_dw,Voc_up,Jsc_up,JMax_up,Vmax_up,PCE_up,FF_up,HI,SolarRad
count,2872,2872.000000,2872.000000,2872.000000,2872.000000,2.854000e+03,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2.854000e+03,2872.000000,2872.000000,2854.000000
mean,2024-10-29 16:22:39.788301056,1.553227,0.860698,0.599647,1.029589,9.254513e+00,48.360523,1.542430,0.903058,0.645130,1.013562,9.510611e+00,84.182679,2.071761,338.649615
min,2024-10-21 06:00:27,0.010000,0.000007,0.000003,0.010000,1.706572e-08,13.691733,0.010000,0.000004,0.000003,0.010000,1.672413e-08,8.245759,1.004958,1.000000
25%,2024-10-25 16:17:43.500000,0.930000,0.108000,0.078099,0.653500,3.620157e-02,42.737428,0.930000,0.114000,0.084818,0.598500,3.563756e-02,43.904748,1.998227,1.000000
50%,2024-10-29 15:25:18.500000,1.096000,0.485235,0.345000,0.709000,2.460983e-01,46.666562,1.096000,0.524500,0.391500,0.653500,2.713602e-01,48.088837,2.016006,217.000000
75%,2024-11-02 14:32:53.249999872,2.479000,1.237861,0.844074,1.594000,2.009143e+00,50.473637,2.479000,1.351370,0.986250,1.594000,2.080244e+00,50.947802,2.239006,643.750000
max,2024-11-06 13:40:29,2.589500,4.043309,2.624198,1.815000,3.019258e+02,517.171634,2.589500,3.811556,2.851012,2.534000,2.989942e+02,25340.000000,5.581992,998.000000
std,NaN,0.818104,0.979012,0.675472,0.540018,3.166757e+01,17.182647,0.837067,0.986833,0.684113,0.563589,3.180223e+01,928.128324,0.324664,339.765191


In [16]:
print(df_combined["SolarRad"].isna().sum())     #NaN de la tormenta: from 2024-08-02 01:38:00 to 2024-08-02 04:16:00

18


In [8]:
##  Testing Data for Nov 5th  ##

specific_date = pd.to_datetime('2024-11-05').date()

# Filter rows for the specific day (ignore time)
filtered_df = df_combined[df_combined['Date'].dt.date == specific_date]
filtered_df.iloc[35:40]               #around 9 
filtered_df.iloc[70:76]               #around 12
filtered_df.iloc[106:112]             #around 15            #Error en el FF_up celda 2 (Corriente cte, ver txt)
#filtered_df.iloc[142:148]             #around 18

Date  Voc_dw    Jsc_dw   Jmax_dw  Vmax_dw  \
Ciclo Celda                                                            
1352  1     2024-11-05 14:50:10   1.041  0.871000  0.573000   0.6535   
      2     2024-11-05 14:50:29   2.534  2.941284  2.013037   1.5385   
1353  1     2024-11-05 15:00:10   1.041  0.829000  0.534000   0.6535   
      2     2024-11-05 15:00:28   2.479  2.862667  1.960198   1.5385   
1354  1     2024-11-05 15:10:09   1.041  0.802000  0.512000   0.6535   
      2     2024-11-05 15:10:28   2.479  2.757432  1.843210   1.5940   

               PCE_dw      FF_dw  Voc_up    Jsc_up   JMax_up  Vmax_up  \
Ciclo Celda                                                             
1352  1      0.042649  41.298220   1.041  1.127000  0.839000   0.5985   
      2      0.352740  41.553317   2.534  2.934815  2.008988   1.5385   
1353  1      0.038817  40.437248   1.041  1.084000  0.807000   0.5985   
      2      0.335458  42.496193   0.010  0.051852  0.051852   2.4790   
1354  1      0.036448  40.076562   1.041  1.062000  0.789000   0.5985   
      2      0.320052  42.981518   2.479  2.771210  1.911704   1.5385   

               PCE_up         FF_up        HI  SolarRad  
Ciclo Celda                                              
1352  1      0.057192     42.800759  2.464223     878.0  
      2      0.352030     41.561140  1.997988     878.0  
1353  1      0.053725     42.801371  2.511236     899.0  
      2      0.014298  24790.000000  1.026452     899.0  
1354  1      0.051440     42.713574  2.541016     918.0  
      2      0.320387     42.812653  2.037160     918.0

In [9]:
df_combined.to_pickle("N_ciclos_AWS.pkl")

In [10]:
df_combined.to_csv("something.csv")